In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/lankenau/isotools/src')

In [3]:
import os
import logging
from isotools import Gene
from isotools import Transcriptome
from isotools import __version__ as isotools_version

# set up logging
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
logger = logging.getLogger("isotools")
logger.info(f"This is isotools version {isotools_version}")

INFO:This is isotools version 0.3.5rc11


In [ ]:
basepath = "/path/to/code/snakemake-pipeline/"
version = 2
tissue = 'brain'
path = os.path.join(basepath, f"results/isotools_v{version}/pkl/{tissue}.pkl")
isoseq = Transcriptome.load(path)

In [5]:
classification = os.path.join(basepath, 'results/sqanti/isotools_v{version}/qc/{tissue}/brain_classification.txt')
isoseq.import_sqanti_classification(classification.format(version=version, tissue=tissue))

100%|██████████| 55811/55811 [00:24<00:00, 2296.77it/s]


In [6]:
isoseq.add_filter(tag='CAGE_SUPPORT', expression='sqanti_classification is not None and sqanti_classification["within_CAGE_peak"]', context='transcript', update=True)
isoseq.add_filter(tag='TSS_RATIO', expression='sqanti_classification is not None and sqanti_classification["ratio_TSS"] > 1.5', context='transcript', update=True)
isoseq.add_filter(tag='POLYA_SITE', expression='sqanti_classification is not None and sqanti_classification["within_polyA_site"]', context='transcript', update=True)
isoseq.add_filter(tag='POLYA_MOTIF', expression='sqanti_classification is not None and sqanti_classification["polyA_motif_found"]', context='transcript', update=True)

isoseq.add_filter(tag='SQANTI_SUPPORT', expression='CAGE_SUPPORT and TSS_RATIO and POLYA_SITE and POLYA_MOTIF', context='transcript', update=True)

In [7]:
next(isoseq.iter_transcripts(query='SQANTI_SUPPORT'))

(<isotools.gene.Gene object at 0x7f8db35db6b0>,
 0,
 {'exons': [[np.int64(108876985), 108877228],
   [108885506, 108885578],
   [108896863, 108897085],
   [108897491, 108897627],
   [108897958, 108898101],
   [108898641, 108898765],
   [108898878, 108898994],
   [108901789, 108901945],
   [108903125, 108903234],
   [108904124, 108904254],
   [108914337, 108914408],
   [108918612, 108918789],
   [108922416, 108922576],
   [108923999, 108924214],
   [108929700, np.int64(108930395)]],
  'strand': '+',
  'coverage': {'ENCSR700EBI': 10, 'ENCSR425HFS': 1, 'ENCSR463IDK': 10},
  'TSS': {'ENCSR700EBI': {108876981: 1,
    108876984: 1,
    108876985: 2,
    108876986: 1,
    108876988: 2,
    108876990: 1,
    108877106: 1,
    108877146: 1},
   'ENCSR425HFS': {108876984: 1},
   'ENCSR463IDK': {108876984: 5,
    108876985: 1,
    108876987: 2,
    108877098: 1,
    108877168: 1}},
  'PAS': {'ENCSR700EBI': {108930396: 7,
    108931627: 1,
    108930404: 1,
    108930391: 1},
   'ENCSR425HFS': {10

In [8]:
count = 0
for transcript in isoseq.iter_transcripts(query='SQANTI_SUPPORT'):
    count += 1
count

11661

In [9]:
coordination_results = isoseq.coordination_test(
    event_type=["TSS", "PAS"],
    min_total=50,
    min_alt_fraction=.05,
    transcript_filter="SQANTI_SUPPORT",
)
coordination_results

,gene_id,gene_name,strand,eventA_type,eventB_type,eventA_start,eventA_end,eventB_start,eventB_end,pvalue,...,dcPSI_AB,dcPSI_BA,priA_priB,priA_altB,altA_priB,altA_altB,priA_priB_transcript_ids,priA_altB_transcript_ids,altA_priB_transcript_ids,altA_altB_transcript_ids
0,ENSG00000142875.21,PRKACB,+,TSS,PAS,84078084,84078371,84204481,84205294,8.387972e-15,...,0.458757,0.491525,29,2,0,28,"[4, 3, 2]",[0],[],[1]
1,ENSG00000142875.21,PRKACB,+,TSS,PAS,84078084,84078371,84235179,84238491,8.387972e-15,...,-0.458757,-0.443958,0,28,29,2,[],[1],"[4, 3, 2]",[0]
2,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164341,84164422,84204481,84205294,5.356978e-02,...,-0.474576,-0.084746,26,5,28,0,"[4, 0]","[3, 2]",[1],[]
3,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164341,84164422,84235179,84238491,5.356978e-02,...,0.474576,0.076545,28,0,26,5,[1],[],"[4, 0]","[3, 2]"
4,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164694,84165013,84204481,84205294,1.225092e-10,...,-0.474576,-0.406780,7,24,28,0,"[3, 0, 2]",[4],[1],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,ENSG00000172465.14,TCEAL1,+,TSS,PAS,103629476,103629573,103629884,103630946,3.526644e-06,...,0.607843,0.334314,31,0,9,11,[1],[],"[2, 0, 4]",[3]
250,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585624,103586648,103587725,1.274413e-63,...,-0.107557,-0.497652,8,570,84,50,[6],[1],"[2, 5]","[3, 0, 4]"
251,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585624,103586653,103587725,1.274413e-63,...,0.107557,0.115373,84,50,8,570,"[2, 5]","[3, 0, 4]",[6],[1]
252,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585683,103586648,103587725,1.742996e-59,...,0.720889,0.473419,570,8,54,80,[1],[6],"[3, 5, 0, 4]",[2]


In [11]:
coordination_results = isoseq.coordination_test(
    event_type=["TSS", "PAS"],
)
coordination_results

,gene_id,gene_name,strand,eventA_type,eventB_type,eventA_start,eventA_end,eventB_start,eventB_end,pvalue,...,dcPSI_AB,dcPSI_BA,priA_priB,priA_altB,altA_priB,altA_altB,priA_priB_transcript_ids,priA_altB_transcript_ids,altA_priB_transcript_ids,altA_altB_transcript_ids
0,ENSG00000121957.15,GPSM2,+,TSS,PAS,108876985,108877228,108929700,108930395,1.343388e-12,...,0.194181,0.204821,23,8,4,69,"[18, 19, 31, 32, 36, 14, 15, 17, 21, 23, 29, 30]","[33, 2, 5, 7, 9, 12, 26]","[16, 34, 35, 28]","[0, 1, 3, 6, 4, 8, 10, 11, 13, 24, 25, 27]"
1,ENSG00000116685.17,KIAA2013,-,TSS,PAS,11925204,11926407,11922207,11923489,1.000000e+00,...,0.000422,0.002194,25,190,5,39,"[13, 12, 8, 32, 33, 34, 35, 6, 7, 10, 14, 15, ...","[2, 4, 24, 1, 9, 25, 28, 29, 30]","[19, 20, 21, 22, 23]","[11, 36]"
2,ENSG00000116685.17,KIAA2013,-,TSS,PAS,11925204,11926407,11920066,11920332,2.571443e-01,...,0.016773,0.015677,20,43,59,186,"[18, 0, 19, 20, 21, 22, 23]","[11, 24, 1, 36, 30]","[3, 13, 12, 8, 32, 33, 34, 35, 6, 7, 10, 14, 1...","[2, 4, 9, 25, 28, 29]"
3,ENSG00000116685.17,KIAA2013,-,TSS,PAS,11922635,11923140,11920066,11920332,3.339780e-03,...,0.175974,0.025139,62,1,211,34,"[11, 18, 24, 1, 36, 19, 20, 21, 22, 23, 30]",[0],"[2, 4, 13, 12, 8, 6, 7, 9, 10, 14, 15, 16, 17,...",[3]
4,ENSG00000117450.15,PRDX1,-,TSS,PAS,45521828,45521883,45511051,45511414,0.000000e+00,...,0.206992,0.195244,387,1,85,1395,"[4, 1, 0, 2, 17]",[19],"[6, 13, 11, 10, 7, 8, 16, 5, 12, 9, 21]","[3, 14, 15, 20, 18, 22]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,ENSG00000134597.17,RBMX2,+,TSS,PAS,130402015,130402370,130412360,130413340,1.036732e-04,...,-0.208498,-0.245921,51,42,35,4,"[1, 4, 6, 7, 9]","[3, 5, 10]",[2],[0]
2572,ENSG00000102125.17,TAFAZZIN,+,TSS,PAS,154411543,154411952,154420902,154421722,3.044157e-02,...,0.033883,0.028418,14,17,47,139,"[52, 96, 98, 4, 50, 83, 91, 28, 94, 95]","[1, 32, 68, 44, 45, 46, 77, 81, 82, 19, 24, 90...","[57, 53, 58, 48, 18, 21, 31, 35, 36, 49, 51, 5...","[5, 3, 8, 9, 14, 0, 13, 23, 27, 64, 33, 2, 6, ..."
2573,ENSG00000292339.1,ASMTL,-,TSS,PAS,1452747,1452925,1403140,1403489,2.130257e-01,...,-0.016087,-0.015938,0,14,15,93,[],"[10, 15, 9, 12]","[5, 2, 4, 3, 11, 13, 14, 16]","[0, 1, 6, 8]"
2574,ENSG00000292358.1,GTPBP6,-,TSS,PAS,318438,318758,304757,305197,1.866802e-03,...,0.047279,0.047279,13,20,20,127,"[3, 2, 34, 6, 8, 9, 13, 46]","[1, 0, 32, 33, 35, 5, 7, 10, 11, 12, 45, 14, 2...","[21, 19, 23, 42, 44, 15, 17, 25]","[16, 18, 38, 20, 31, 37, 39, 40, 41, 43, 22, 2..."
